<a href="https://colab.research.google.com/github/gmauricio-toledo/SA-paper/blob/master/SMS-experiments-borrar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importar los archivos del proyecto

In [ ]:
!python -m spacy download es_core_news_sm

## Opción 1: Traer sólo los módulos con las clases:

In [11]:
import requests

def get_module(url):
    fname = url.split('/')[-1]
    r = requests.get(url)
    with open(fname, 'w') as f:
        f.write(r.text)

In [ ]:
urls = ["https://raw.githubusercontent.com/gmauricio-toledo/SA-paper/master/experiment_tools_SA.py",
        "https://raw.githubusercontent.com/gmauricio-toledo/SA-paper/master/scoring.py",
        "https://raw.githubusercontent.com/gmauricio-toledo/SA-paper/master/SentimentKW.py",
        "https://raw.githubusercontent.com/gmauricio-toledo/SA-paper/master/TextCleaner.py",
        "https://raw.githubusercontent.com/gmauricio-toledo/SA-paper/master/TextRank.py"
        ]

for url in urls:
    get_module(url)

## Opción 2: Traer todo el repositorio github en el root

In [1]:
!git init
!git pull https://github.com/gmauricio-toledo/SA-paper.git 

Initialized empty Git repository in /content/.git/


In [13]:
!pip install --upgrade gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 24.1 MB 52.7 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


# Experiments

In [1]:
from gensim.models import Word2Vec
from scoring import Scoring
import pandas as pd
import numpy as np

## Single run

In [2]:
import pandas as pd
from gensim.models import Word2Vec

w2v_model = Word2Vec.load("data/word2vec.model")

# ---- Leemos los mensajes ----
msjs_df = pd.read_csv("data/sms_full_dataframe.csv",index_col=0)
# msjs_df['Normalized Label'] = (2/5)*msjs_df['ds'].values-1
# y = msjs_df['ds'].values

msjs_df.sample(3)

,Llave,Paciente,Sexo,cal_vida,Texto,ds03,ds45,dp0,dp1,ds0,ds1,ds2,ds3,ds4,ds5,ds,dp
3773,16019,7959,1,46.6,sigo soñando muchas personas cada vez definida...,0,1,1,0.0,0,0,0,0,0,1,5,0.0
4827,18118,11770,1,68.0,semana bastante estresante pensando jueves cit...,0,1,1,0.0,0,0,0,0,0,1,5,0.0
4186,16810,9282,1,65.2,hoy ido excursión unas amigas agencia arawak v...,0,1,0,1.0,0,0,0,0,1,0,4,1.0


In [3]:
from experiment_tools_SA import SentimentAnalysis


# betas1 = [0.5,1,2]#np.linspace(0.1,2,5)
# betas2 = [0.5,1,2]#np.linspace(0.5,5,5)
# nums_cols = [3,5,7]
# alphas = [0.5,0.75,0.9]#np.linspace(0.25,0.98,5)

# params_dict = {
#                 # 'beta1':betas1,
#                 # 'beta2':betas2,
#                 'n_cols':nums_cols
#                 # 'alpha':alphas
#                 }



hpd = {'emb_model': w2v_model}

sa = SentimentAnalysis(hyper_params_dict=hpd,
                        df=msjs_df,
                        text_col_name='Texto',
                        label_col_name='ds',
                        )

Dataframe contains entries with 6 labels:
[0 1 2 3 4 5]


In [4]:
comb_dict = {
            'beta1': 1,
            'beta2': 5,
            'n_cols': 9,
            'alpha': 0.9,
            'top_n': 100,
            'n_iter': 3
            }

results = sa.run(combination_dict=comb_dict)

Calculando palabras prototípicas... done in 381.64676
Construyendo vecinos... done in 337.2408
Calculando puntajes... done in 8.02782
Calculando representaciones... done in 5.04637
43/43 [==============================] - 0s 9ms/step - loss: 1.3738 - accuracy: 0.5237
done in 27.65698
Iteración 1/3 completada.
Construyendo vecinos... done in 735.70558
Calculando puntajes... done in 8.03441
Calculando representaciones... done in 4.92504
43/43 [==============================] - 0s 9ms/step - loss: 1.2090 - accuracy: 0.5666
done in 33.54966
Iteración 2/3 completada.
Construyendo vecinos... done in 736.01638
Calculando puntajes... done in 8.07116
Calculando representaciones... done in 5.15763
43/43 [==============================] - 0s 9ms/step - loss: 1.2867 - accuracy: 0.5470
done in 32.50956
Iteración 3/3 completada.


In [6]:
print(results)

{'test_loss': 1.286723256111145, 'test_accuracy': 0.5469774007797241, 'predictions': array([5, 5, 5, ..., 5, 5, 0])}


## Gridsearch

In [5]:
from experiment_tools_SA import SentimentAnalysis
import pandas as pd
from gensim.models import Word2Vec

w2v_model = Word2Vec.load("data/word2vec.model")

# ---- Leemos los mensajes ----
msjs_df = pd.read_csv("data/sms_full_dataframe.csv",index_col=0)
display(msjs_df.sample(3))

hpd = {'emb_model': w2v_model}

sa = SentimentAnalysis(hyper_params_dict=hpd,
                        df=msjs_df,
                        text_col_name='Texto',
                        label_col_name='ds',
                        )

,Llave,Paciente,Sexo,cal_vida,Texto,ds03,ds45,dp0,dp1,ds0,ds1,ds2,ds3,ds4,ds5,ds,dp
2892,13973,5924,1,77.2,hoy ido gimnasio bien cómoda tarde ido dar pas...,0,1,1,0.0,0,0,0,0,0,1,5,0.0
504,3866,1167,1,16.8,dormido bien único recogido bajado perro menud...,1,0,0,1.0,0,0,1,0,0,0,2,1.0
4093,16620,8857,2,26.6,hola rodado águila roja carlos emperador espec...,0,1,0,1.0,0,0,0,0,1,0,4,1.0


Dataframe contains entries with 6 labels:
[0 1 2 3 4 5]


In [ ]:
betas1 = [0.5,1,2] #np.linspace(0.1,2,5)
betas2 = [0.5,1,2] #np.linspace(0.5,5,5)
nums_cols = [3,5,7,9]
alphas = [0.5,0.75,0.9]#np.linspace(0.25,0.98,5)
n_iters = [3,4,5]
tops_n = [50,100,200]

params_dict = {
                'beta1':betas1,
                'beta2':betas2,
                'n_cols':nums_cols,
                'alpha':alphas
                }


dpd = {
                'beta1': 1,
                'beta2': 1,
                'n_cols': 5,
                'alpha': 0.5,
                'top_n': 50,
                'n_iter': 3
                }

sa.grid_search(param_dict=params_dict,
                default_params_dict=dpd)

In [7]:
sa.best_accuracy

0.6023306846618652

In [8]:
sa.best_params_dict

{'beta1': 0.5, 'beta2': 2, 'n_cols': 5, 'alpha': 0.9, 'top_n': 50, 'n_iter': 3}